# Exercise 1: Data cleaning

Before doing actual data analysis, we usually first need to clean the data. 
This might involve steps such as dealing with missing values and encoding categorical variables as integers.
In this exercise, you will perform such steps based on the Titanic passenger data.

1. Load the Titanic data set in `titanic.csv` located in the `data/` folder.
2. Report the number of observations with missing `Age`, for example using [`isna()`](https://pandas.pydata.org/docs/reference/api/pandas.isna.html).
3. Compute the average age in the data set. Use the following approaches and compare your results:
    1.  Use pandas's [`mean()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mean.html) method.
    2.  Convert the `Age` column to a NumPy array using [`to_numpy()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_numpy.html). Experiment with NumPy's [`np.mean()`](https://numpy.org/doc/2.0/reference/generated/numpy.mean.html) and [`np.nanmean()`](https://numpy.org/doc/2.0/reference/generated/numpy.nanmean.html) to see if you obtain the same results.
4. Replace the all missing ages with the mean age you computed above, rounded to the nearest integer.
   Note that in "real" applications, replacing missing values with sample means is usually not a good idea.
5. Convert this updated `Age` column to integer type using [`astype()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html).
6. Generate a new column `Female` which takes on the value one if `Sex` is equal to `"female"` and zero otherwise. 
   This is called an _indicator_ or _dummy_ variable, and is preferrable to storing such categorical data as strings.
   Delete the original column `Sex`.
7. Save your cleaned data set as `titanic-clean.csv` using [`to_csv()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html) with `,` as the field separator.
   Tell `to_csv()` to *not* write the `DataFrame` index to the CSV file as it's not needed in this example.

In [46]:
import pandas as pd
import numpy as np

In [65]:
DATA_PATH = '../../data'
file = f'{DATA_PATH}/titanic.csv'
df=pd.read_csv(file, sep = ',')
isnone = df['Age'].isna().sum()
print(f'The number of missing observations in Age is {isnone}')
meandf = round(df['Age'].mean(), 1)
print(f'The average age is {meandf}')
dfnumpy = df['Age'].to_numpy()
dfnumpymean = np.mean(dfnumpy)
print(f'The average age (with mean numpy) is {dfnumpymean}')
dfnumpynanmean = round(np.nanmean(dfnumpy),1)
print(f'The average age (using nanmean numpy) is {dfnumpynanmean}')
dffilled = df['Age'].fillna(meandf)
dffilledint = dffilled.astype(int)
df['Age'] = dffilledint
df['Female'] = (df['Sex']=='female').astype(int)

del df['Sex']

The number of missing observations in Age is 177
The average age is 29.7
The average age (with mean numpy) is nan
The average age (using nanmean numpy) is 29.7


In [68]:
df

df.to_csv('titanic-clean.csv', sep = ',', index = False)

***
# Exercise 2: Selecting subsets of data

In this exercise, you are asked to select subsets of macroeconomic data for the United States based on some criteria.

1.  Load the annual data from FRED which are located in `FRED_annual.xlsx` 
    in the `data/FRED` folder.

2.  Print the list of columns and the number of non-missing observations.

3.  Since we are dealing with time series data, set the column `Year` as the DataFrame index.

4.  Print all observations for the 1960s decade using at least two different methods.

5.  Using the data in the column `GDP`, compute the annual GDP growth in percent and store it in the column `GDP_growth`. Select the years in which

    1.  GDP growth was above 5%.
    2.  GDP growth was negative, but inflation as still above 5% (such episodes are called "stagflation" since usually negative GDP growth is associated with low inflation).
        
        Use at least two methods to select such years.

    *Hint:* You can compute changes relative to the previous observation using the 
    [`pct_change()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pct_change.html) method.


In [113]:
fredfile = f'{DATA_PATH}/FRED/FRED_annual.xlsx'
dffred = pd.read_excel(fredfile)
collist = dffred.columns.tolist()
print(collist)
nonmissing = dffred.notna().sum()
print(nonmissing)
dffred = dffred.set_index('Year')
dffred.loc["1960":"1970"]
dffred['GDP_growth']= dffred['GDP'].pct_change()
dffred.loc[(dffred['GDP_growth']>0.05) | ((dffred['GDP_growth']<0)&(dffred['INFLATION']>5))]



['Year', 'GDP', 'CPI', 'UNRATE', 'FEDFUNDS', 'INFLATION']
Year         70
GDP          70
CPI          70
UNRATE       70
FEDFUNDS     70
INFLATION    69
dtype: int64


,GDP,CPI,UNRATE,FEDFUNDS,INFLATION,GDP_growth
Year,,,,,,
1955,3083.0,26.8,4.4,1.8,-0.371747,0.071342
1959,3412.4,29.2,5.4,3.3,1.038062,0.069316
1962,3810.1,30.3,5.6,2.7,1.337793,0.061280
1964,4205.3,31.0,5.2,3.5,1.307190,0.057644
1965,4478.6,31.5,4.5,4.1,1.612903,0.064989
1966,4773.9,32.5,3.8,5.1,3.174603,0.065936
1972,5780.0,41.8,5.6,4.4,3.209877,0.052555
1973,6106.4,44.4,4.9,8.7,6.220096,0.056471
1974,6073.4,49.3,5.6,10.5,11.036036,-0.005404


***
# Exercise 3: Labor market statistics for the US

In this exercise, you are asked to compute some descriptive statistics for the unemployment rate 
and the labor force participation (the fraction of the working-age population in the labor force, i.e., individuals who are either employed or unemployed) for the United States.

1.  Load the monthly time series from FRED which are located in `FRED_monthly.csv` 
    in the `data/FRED` folder.

    *Hint:* You can use `pd.read_csv(..., parse_dates=['DATE'])` to automatically
    parse strings stored in the `DATE` column as dates.

2.  Print the list of columns and the number of non-missing observations.

3.  Since we are dealing with time series data, set the column `DATE` as the DataFrame index. Using the date index, select all observations from the first three months
    of the year 2020.

4.  For the columns `UNRATE` (unemployment rate) and `LFPART` (labor force participation), compute and report 
    the mean, minimum and maximum values for the whole sample. Round your results 
    to one decimal digit.

    *Hint:* You can use the DataFrame methods 
    [`mean()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mean.html),
    [`min()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.min.html), and 
    [`max()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.max.html)
    to compute the desired statistics.

    *Hint:* You can use the DataFrame method
    [`round()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.round.html)
    to truncate the number of decimal digits.

5.  You are interested in how the average unemployment rate evolved over the last 
    few decades. 

    -   Add a new column `Decade` to the DataFrame which contains the 
        starting year for each decade (e.g., this value should be 1950
        for the years 1950-1959, and so on).

        *Hint:* The decade can be computed from the column `Year` using 
        truncated integer division:
        ```python
        df['Year'] // 10 * 10
        ```

    -   Write a loop to compute and report the average unemployment rate (column `UNRATE`) 
        for each decade.

        Include only the decades from 1950 to 2010 for which you have all
        observations. 

***
# Exercise 4: Working with string data (advanced)

Most of the data we deal with contain strings, i.e., text data (names, addresses, etc.). Often, such data is not in the format needed for analysis, and we have to perform additional string manipulation to extract the exact data we need. This can be achieved using the pandas [string methods](https://pandas.pydata.org/docs/user_guide/text.html#string-methods).

To illustrate, we use the Titanic data set for this exercise.

1.  Load the Titanic data and restrict the sample to men. (This simplifies the task. Women in this data set have much more complicated names as they contain both their husband's and their maiden name)
2.  Print the first five observations of the `Name` column. As you can see, the data is stored in the format _"Last name, Title First name"_ where title is something like Mr., Rev., etc.
3. Split the `Name` column by `,` to extract the last name and the remainder as separate columns. You can achieve this using the [`partition()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.partition.html#pandas.Series.str.partition) string method.
4. Split the remainder (containing the title and first name) using the space character `" "` as separator to obtain individual columns for the title and the first name.
5. Store the three data series in the original `DataFrame` (using the column names `FirstName`, `LastName` and `Title`) and delete the `Name` column which is no longer needed.
6. Finally, extract the ship deck from the values in `Cabin`. The ship deck is the first character in the string stored in `Cabin` (A, B, C, ...). You extract the first character using the 
[`get()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.get.html#pandas.Series.str.get) string method. Store the result in the column `Deck`.

*Hint*: Pandas's string methods can be accessed using the `.str` attribute. For example, to partition values in the column `Name`, you need to use
```python
df['Name'].str.partition()
```
